In [92]:
import requests
import json
from pprint import pprint
import random
import pandas as pd
import gmaps

from citipy import citipy
from api_keys import api_key

In [84]:
latitudes = []
longitudes =[]
coordinates = []

In [85]:
# Get random coordinates from each quadrant.

#1st quadrant
latitudes = [random.uniform(0, 90) for x in range(2)]  ## change 2 to 125!!!!!!!!!!!!!!!!!!!1
longitudes = [random.uniform(0, -180) for x in range(2)]
first_coordinates = list(zip(latitudes, longitudes))

#2nd quadrant
latitudes = [random.uniform(0, 90) for x in range(2)]
longitudes = [random.uniform(0, 180) for x in range(2)]
second_coordinates =(list(zip(latitudes, longitudes)))

#3rd quadrant
latitudes = [random.uniform(0, -90) for x in range(2)]
longitudes = [random.uniform(0, -180) for x in range(2)]
third_coordinates = list(zip(latitudes, longitudes))
              
#4th quadrant
latitudes = [random.uniform(0, -90) for x in range(2)]
longitudes = [random.uniform(0, 180) for x in range(2)]
fourth_coordinates = list(zip(latitudes, longitudes))
 
coordinates = first_coordinates + second_coordinates + third_coordinates + fourth_coordinates

In [86]:
url = "http://api.openweathermap.org/data/2.5/weather?"
unit = "imperial"

In [87]:
non_repeat_city_list = {}

max_temperature_list = []
humidity_list = []
cloudiness_list = []
city_list = []
country_list = []
wind_speed_list = []
latitude_list = []
longitude_list = []

In [88]:
# Pass coordinates to citipy to find closest city.
num = 0

for lat, lon in coordinates:
    nearest_city = citipy.nearest_city(lat, lon)
    
    city = nearest_city.city_name
    country = nearest_city.country_code
    
#     if city == '':
#         print("City not found. Skipping city")
#         continue
    
    if city not in non_repeat_city_list:
        query_url = f"{url}APPID={api_key}&units={unit}&q={city}&country={country}"
        response = requests.get(query_url).json()
        
        if 'id' not in response:
            print("City not found. Skipping city")
            continue
        
        num += 1
        city_number = response['id']
        print(f"Request {num} - city#: {city_number}; city: {city}, country: {country}")
              
        # Capture city info from openweathermap, as well as actual latitude and longitude of closest city.
        max_temperature = response['main']['temp_max']
        max_temperature_list.append(max_temperature)
        
        humidity = response['main']['humidity']
        humidity_list.append(humidity)
        
        cloudiness = response['clouds']['all']
        cloudiness_list.append(cloudiness)
        
        wind_speed = response['wind']['speed']
        wind_speed_list.append(wind_speed)
        
        actual_latitude = response['coord']['lat']
        latitude_list.append(actual_latitude)
        
        actual_longitude = response['coord']['lon']
        longitude_list.append(actual_longitude)
        
        city_list.append(city)
        country_list.append(country.upper())
        
        non_repeat_city_list[city] = country, max_temperature, humidity, cloudiness, wind_speed, actual_latitude, actual_longitude
     
print(non_repeat_city_list) 

Request 1 - city#: 4488762; city: rocky mount, country: us
City not found. Skipping city
Request 2 - city#: 1862505; city: hirara, country: jp
Request 3 - city#: 544318; city: kosa, country: ru
Request 4 - city#: 6201424; city: mataura, country: pf
Request 5 - city#: 1273574; city: vaini, country: to
Request 6 - city#: 2206894; city: hokitika, country: nz
Request 7 - city#: 4301307; city: morehead, country: pg
{'rocky mount': ('us', 72, 30, 1, 17.22, 35.94, -77.8), 'hirara': ('jp', 72.94, 100, 32, 14.47, 24.8, 125.28), 'kosa': ('ru', 34.69, 91, 92, 11.56, 59.94, 54.99), 'mataura': ('pf', 48.99, 95, 92, 2.95, -46.19, 168.86), 'vaini': ('to', 74.56, 74, 0, 3.51, 15.34, 74.49), 'hokitika': ('nz', 51.52, 100, 0, 7.87, -42.72, 170.96), 'morehead': ('pg', 43, 60, 90, 6.93, 38.18, -83.43)}


In [89]:
city_df = pd.DataFrame({
    "City": city_list,
    "Country": country_list,
    "Max Temperature": max_temperature_list,
    "Humidity": humidity_list,
    "Cloudiness": cloudiness_list,
    "Wind Speed": wind_speed_list,
    "Latitude": latitude_list,
    "Longitude": longitude_list
})
city_df

,City,Country,Max Temperature,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
0,rocky mount,US,72.00,30,1,17.22,35.94,-77.80
1,hirara,JP,72.94,100,32,14.47,24.80,125.28
2,kosa,RU,34.69,91,92,11.56,59.94,54.99
3,mataura,PF,48.99,95,92,2.95,-46.19,168.86
4,vaini,TO,74.56,74,0,3.51,15.34,74.49
5,hokitika,NZ,51.52,100,0,7.87,-42.72,170.96
6,morehead,PG,43.00,60,90,6.93,38.18,-83.43


In [91]:
print(len(city_df))

7


In [94]:
fig = gmaps.figure()
fig

Figure(layout=FigureLayout(height='420px'))